In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-dataset/modified_dataset.csv
/kaggle/input/modified-dataset/modified_dataset.csv
/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv


In [2]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer, GPT2Model,GPT2Tokenizer,GPT2LMHeadModel
from transformers import AutoTokenizer, T5ForConditionalGeneration , T5Tokenizer , T5Model
import pandas as pd

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
checkpoint = "t5-base"

In [7]:
t5_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
#set the max length to model's default present max length
t5_tokenizer.model_max_length = t5_tokenizer.model_max_length

In [9]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [10]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [11]:
file_path = "/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv"
df = pd.read_csv(file_path)

In [12]:
file_path = "/kaggle/input/modified-dataset/modified_dataset.csv"
df1 = pd.read_csv(file_path)

In [13]:
df1.columns

Index(['description', 'student_code', 'feedback', 'metacognitive_feedback',
       'metacognitive_profile'],
      dtype='object')

In [14]:
df.head(5)

,question,difficulty,prefer_solution,flaw_solution,random_col_1,random_col_2,random_col_3,random_col_4,random_col_5,random_col_6,...,random_col_9,random_col_10,random_col_11,random_col_12,random_col_13,random_col_14,random_col_15,random_col_16,metacognitive_vector,metacognitive_feedback
0,"=====Problem Statement=====\nGiven an integer,...",introductory,n = int(input().strip())\nw = len(str(bin(n))[...,w = len(str(bin(n))[2:])\nn = int(input().stri...,2,2,1,2,2,2,...,1,1,1,1,3,1,2,2,"[2, 2, 1, 2, 2, 2, 3, 2, 1, 1, 1, 1, 3, 1, 2, 2]","Your solution is very close to being correct, ..."
1,=====Problem Statement=====\nYou are given a s...,introductory,#!/usr/bin/env python3\n\ndef __starting_point...,#!/usr/bin/env python3\n\ndef __starting_point...,1,3,1,2,2,2,...,2,3,1,3,3,1,2,1,"[1, 3, 1, 2, 2, 2, 3, 2, 2, 3, 1, 3, 3, 1, 2, 1]",It's clear that you have a good understanding ...
2,=====Problem Statement=====\nYou are given a s...,introductory,A = set(input().split())\nn = int(input())\nc...,s = set(input().split())\nm = int(input())\nc...,1,1,2,3,3,3,...,1,3,1,1,1,2,1,1,"[1, 1, 2, 3, 3, 3, 2, 2, 1, 3, 1, 1, 1, 2, 1, 1]",Your approach to the problem shows a good unde...
3,=====Problem Statement=====\nYou are given a p...,introductory,"for i in range(1,int(input())): #More than 2 l...","for idx in range(1, int(input())):\n print(...",2,1,1,3,1,3,...,1,3,1,1,3,1,3,2,"[2, 1, 1, 3, 1, 3, 1, 2, 1, 3, 1, 1, 3, 1, 3, 2]",Your approach to the problem shows a good unde...
4,=====Function Descriptions=====\nsum\n\nThe su...,introductory,"import numpy\nn,m=list(map(int,input().split()...",Variable Renaming\n\n```\nimport numpy as np\n...,2,2,3,2,2,1,...,1,2,3,3,3,1,3,2,"[2, 2, 3, 2, 2, 1, 1, 3, 1, 2, 3, 3, 3, 1, 3, 2]",It appears you are on the right track with you...


In [15]:
df['metacognitive_vector'][0].type

AttributeError: 'str' object has no attribute 'type'

In [16]:
df.columns

Index(['question', 'difficulty', 'prefer_solution', 'flaw_solution',
       'random_col_1', 'random_col_2', 'random_col_3', 'random_col_4',
       'random_col_5', 'random_col_6', 'random_col_7', 'random_col_8',
       'random_col_9', 'random_col_10', 'random_col_11', 'random_col_12',
       'random_col_13', 'random_col_14', 'random_col_15', 'random_col_16',
       'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [17]:
df = df.loc[:, ~df.columns.str.startswith('random_col_')]

In [18]:
df['combined_question_student'] = df['question'] + " " + df['flaw_solution']

In [19]:
df['combined_question_expected'] = df['question'] + " " + df['prefer_solution']

In [20]:
df.columns

Index(['question', 'difficulty', 'prefer_solution', 'flaw_solution',
       'metacognitive_vector', 'metacognitive_feedback',
       'combined_question_student', 'combined_question_expected'],
      dtype='object')

In [21]:
df['metacognitive_feedback'][100]

"It seems you're on the right track with your approach to solving the problem, but there are a few areas where you can improve both your coding skills and your metacognitive strategies. Firstly, ensure you read the problem statement thoroughly and identify all key requirements, inputs, outputs, and constraints. This will help you avoid minor mistakes, such as using incorrect variable names (e.g., using 'm' instead of 'n, l'). Breaking down the problem into smaller, manageable sub-goals can also help. For instance, start by understanding the input format, then focus on sorting the strings, and finally concatenate them. Before coding, try to sketch out your algorithm or plan your solution. This will help you systematically execute the designed algorithm and verify that you're on the correct path to the solution. Pay close attention during the implementation process to avoid negligent mistakes, such as missing colons or incorrect variable assignments. Monitor your problem-solving steps to

In [22]:
df.head(5)

,question,difficulty,prefer_solution,flaw_solution,metacognitive_vector,metacognitive_feedback,combined_question_student,combined_question_expected
0,"=====Problem Statement=====\nGiven an integer,...",introductory,n = int(input().strip())\nw = len(str(bin(n))[...,w = len(str(bin(n))[2:])\nn = int(input().stri...,"[2, 2, 1, 2, 2, 2, 3, 2, 1, 1, 1, 1, 3, 1, 2, 2]","Your solution is very close to being correct, ...","=====Problem Statement=====\nGiven an integer,...","=====Problem Statement=====\nGiven an integer,..."
1,=====Problem Statement=====\nYou are given a s...,introductory,#!/usr/bin/env python3\n\ndef __starting_point...,#!/usr/bin/env python3\n\ndef __starting_point...,"[1, 3, 1, 2, 2, 2, 3, 2, 2, 3, 1, 3, 3, 1, 2, 1]",It's clear that you have a good understanding ...,=====Problem Statement=====\nYou are given a s...,=====Problem Statement=====\nYou are given a s...
2,=====Problem Statement=====\nYou are given a s...,introductory,A = set(input().split())\nn = int(input())\nc...,s = set(input().split())\nm = int(input())\nc...,"[1, 1, 2, 3, 3, 3, 2, 2, 1, 3, 1, 1, 1, 2, 1, 1]",Your approach to the problem shows a good unde...,=====Problem Statement=====\nYou are given a s...,=====Problem Statement=====\nYou are given a s...
3,=====Problem Statement=====\nYou are given a p...,introductory,"for i in range(1,int(input())): #More than 2 l...","for idx in range(1, int(input())):\n print(...","[2, 1, 1, 3, 1, 3, 1, 2, 1, 3, 1, 1, 3, 1, 3, 2]",Your approach to the problem shows a good unde...,=====Problem Statement=====\nYou are given a p...,=====Problem Statement=====\nYou are given a p...
4,=====Function Descriptions=====\nsum\n\nThe su...,introductory,"import numpy\nn,m=list(map(int,input().split()...",Variable Renaming\n\n```\nimport numpy as np\n...,"[2, 2, 3, 2, 2, 1, 1, 3, 1, 2, 3, 3, 3, 1, 3, 2]",It appears you are on the right track with you...,=====Function Descriptions=====\nsum\n\nThe su...,=====Function Descriptions=====\nsum\n\nThe su...


In [23]:
from torch.utils.data import Dataset
import ast
class CustomDataset(Dataset):
    def __init__(self, dataset, t5_tokenizer,gpt2_tokenizer, max_length=512):
        self.t5_tokenizer = t5_tokenizer
        self.gpt2_tokenizer = gpt2_tokenizer
        self.data = dataset
        self.max_length = max_length

        # self.data = data.reset_index(drop=True)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        metacognition_vector = self.data['metacognitive_vector'][idx]
        problem_student_code = self.data['combined_question_student'][idx]
        problem_expected_code = self.data['combined_question_expected'][idx]
        student_code = self.data['flaw_solution'][idx]
        target = self.data['metacognitive_feedback'][idx]

        # Tokenize inputs and truncate/pad
        metacognition_vector_ids = torch.tensor(
            ast.literal_eval(metacognition_vector), dtype=torch.float
        )
        problem_student_code_ids = torch.tensor(
            self.gpt2_tokenizer.encode(problem_student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        problem_expected_code_ids = torch.tensor(
            self.t5_tokenizer.encode(problem_expected_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        # feedback_encoded_ids = torch.tensor(
        #     self.gpt2_tokenizer.encode(feedback, max_length=self.max_length, truncation=True, padding="max_length")
        # )
        
        student_code_ids = torch.tensor(
            self.t5_tokenizer.encode(student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        target_ids = torch.tensor(
            self.gpt2_tokenizer.encode(target, max_length=self.max_length, truncation=True, padding="max_length")
        )

        return metacognition_vector_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids

In [24]:
dataset = CustomDataset(df, t5_tokenizer, gpt2_tokenizer)

In [25]:
dataset1 = CustomDataset(df1, t5_tokenizer, gpt2_tokenizer)

In [26]:
len(dataset)

500

In [27]:
metacognition_vector_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids = dataset[100]
#print(f"Sample {10}:")
print(f"Metacognition vector IDs: {metacognition_vector_ids}")
#print(f"Problem IDs: {description_ids}")
print(f"Expected feedback IDs: {problem_student_code_ids.shape}")
print(f"Expected encoded feedback IDs: {problem_expected_code_ids.shape}")
#print(f"Student Answer IDs: {student_code_ids}")
print(f"Target IDs: {target_ids.shape}")
print("\n")

Metacognition vector IDs: tensor([1., 2., 1., 2., 2., 2., 3., 1., 2., 2., 2., 3., 1., 2., 3., 2.])
Expected feedback IDs: torch.Size([512])
Expected encoded feedback IDs: torch.Size([512])
Target IDs: torch.Size([512])




In [ ]:
# model = T5ForConditionalGeneration.from_pretrained(checkpoint)

In [ ]:
# save_directory = "./t5_checkpoint"
# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory)

In [28]:
gpt2_tokenizer.pad_token

'<|endoftext|>'

In [29]:
gpt2_pad_token_id = gpt2_tokenizer.pad_token_id

In [30]:
gpt2_pad_token_id

50256

In [31]:
t5_tokenizer.pad_token

'<pad>'

In [32]:
t5_pad_token_id = t5_tokenizer.pad_token_id

In [33]:
t5_pad_token_id

0

* context  = problem + expected answer
* current state = problem + student code
* persona = metacognitive vector mapped into linear representaion
or else for persona:
* persona  = metacognitive vector + prompt of metacognition profile defined

# Context Enocder

In [34]:
class ContextEncoder(nn.Module): 
    def __init__(self, t5_model_name='t5-base', output_dim=768): 
        super(ContextEncoder, self).__init__()
        
        self.t5_encoder = T5Model.from_pretrained(t5_model_name).encoder
        self.fc = nn.Linear(self.t5_encoder.config.d_model, output_dim)
    
    def forward(self, problem_student_code_ids, attention_masks=None):        

        encoder_outputs = self.t5_encoder(
            input_ids=problem_student_code_ids,
            attention_mask=attention_masks
        )
        context_hidden_states = encoder_outputs.last_hidden_state   
        
        context_rep = context_hidden_states.mean(dim=1)
        
       
        context_rep = self.fc(context_rep)
        final_rep = context_rep.unsqueeze(1)
        
        return final_rep

In [35]:
context_encoder = ContextEncoder().to(device)

In [36]:
context_encoder

ContextEncoder(
  (t5_encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
              (a

In [37]:
class PersonaEncoder(nn.Module): 
    def __init__(self, t5_model_name='t5-base', output_dim=768): 
        super(PersonaEncoder, self).__init__()
        
        self.t5_encoder = T5Model.from_pretrained(t5_model_name).encoder
        self.fc = nn.Linear(self.t5_encoder.config.d_model, output_dim)
    
    def forward(self, problem_student_code_ids, metacognitive_vector_ids, attention_masks=None):        
        
        combined_inputs = torch.cat([problem_student_code_ids, metacognitive_vector_ids], dim=1)
        print("combine input", combined_inputs.shape)
        
        # if attention_masks is not None:
        #     combined_masks = torch.cat(attention_masks, dim=1)
        # else:
        #     combined_masks = None

        encoder_outputs = self.t5_encoder(
            input_ids=combined_inputs,
            attention_mask=attention_masks
        )
        context_hidden_states = encoder_outputs.last_hidden_state   
        
        context_rep = context_hidden_states.mean(dim=1)
        
       
        context_rep = self.fc(context_rep)
        final_rep = context_rep.unsqueeze(1)
        
        return final_rep

In [ ]:
persona_enocder = PersonaEnocder()

In [38]:
class MetacognitionLayer(nn.Module):
    def __init__(self, metacognitive_dim=16, output_dim=768):
        super(MetacognitionLayer, self).__init__()
        #16 to 768 mapping
        self.metacognitive_fc = nn.Linear(metacognitive_dim, output_dim) 
        self.final_fc = nn.Linear(output_dim, output_dim)

    def forward(self, metacognitive_vector):
        

        metacognitive_rep = self.metacognitive_fc(metacognitive_vector)
        final_rep = self.final_fc(metacognitive_rep)  
        persona_rep = final_rep.unsqueeze(1)

        return persona_rep

In [39]:
metacognitive_emb = MetacognitionLayer().to(device)

In [40]:
metacognitive_emb

MetacognitionLayer(
  (metacognitive_fc): Linear(in_features=16, out_features=768, bias=True)
  (final_fc): Linear(in_features=768, out_features=768, bias=True)
)

In [41]:
class PAALayer(nn.Module):
    def __init__(self, hidden_dimension = 768 , tau=0.8,dropout_rate=0.1):
        super(PAALayer, self).__init__()
        self.hidden_dimenstion = hidden_dimension
        self.tau = tau

       
        self.fc = nn.Linear(2 * hidden_dimension, hidden_dimension)  
        self.sigmoid = nn.Sigmoid()
        self.fc_out = nn.Linear(hidden_dimension, hidden_dimension)
        self.dropout = nn.Dropout(p=dropout_rate)


    def forward(self, hR , oP, oC):
       
        Mp_input  = torch.cat([hR,oP], dim=-1)        
        Mp = self.fc(Mp_input)      
        Wp = self.sigmoid(Mp)      
     
        Mpersona = Wp
        Mcontext = 1 - Wp      
       
        oP_weighted = Mpersona * oP       
        oC_weighted = Mcontext * oC
       
        HPAA = oP_weighted + oC_weighted 
       
        output = self.fc_out(HPAA)
       
        return output

In [42]:
paa = PAALayer()

In [43]:
paa

PAALayer(
  (fc): Linear(in_features=1536, out_features=768, bias=True)
  (sigmoid): Sigmoid()
  (fc_out): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [44]:
class CustomTransformerBlock(nn.Module):
    def __init__(self, hidden_size, tau , dropout_rate=0.1):
        super(CustomTransformerBlock, self).__init__()

        self.self_attention = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)


        self.context_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)
        self.persona_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)

        self.paa_layer = PAALayer(hidden_dimension=hidden_size, tau=tau)
        
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(2048, hidden_size),
            nn.LayerNorm(hidden_size)
        )
        self.layer_norm2 = nn.LayerNorm(hidden_size)
       
    def forward(self, inputs_embeds, encoded_persona, encoded_context):

        hR, _ = self.self_attention(inputs_embeds, inputs_embeds, inputs_embeds)

        oP, _ = self.persona_attn(hR, encoded_persona, encoded_persona)
        oC, _ = self.context_attn(hR, encoded_context, encoded_context)      
        
        HPAA = self.paa_layer(hR, oP, oC)
        
        mlp_output = self.mlp(HPAA)
        output = self.layer_norm2(mlp_output + HPAA)
      
        return output


In [45]:
custom_layer = CustomTransformerBlock(768,0.5)

In [46]:
custom_layer

CustomTransformerBlock(
  (self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (context_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (persona_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (paa_layer): PAALayer(
    (fc): Linear(in_features=1536, out_features=768, bias=True)
    (sigmoid): Sigmoid()
    (fc_out): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=2048, out_features=768, bias=True)
    (4): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=Tr

In [ ]:
class PAAModel(nn.Module):
    def __init__(self, hidden_size=768, vocab_size = 50257 ,tau=0.8, max_length=512, num_transformer_blocks=6):
        super(PAAModel , self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.num_transformer_blocks = num_transformer_blocks

        self.context_encoder = ContextEncoder()
        
        self.metacognitive_emb = MetacognitionLayer()
        
        for param in self.context_encoder.parameters():
            param.requires_grad = False

        for param in self.metacognitive_emb.parameters():
            param.requires_grad = False

        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_length, hidden_size)
        self.dropout = nn.Dropout(p=0.1) 

        self.self_attention = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)        
        self.transformer_blocks = nn.ModuleList([CustomTransformerBlock(hidden_size, tau) for _ in range(num_transformer_blocks)])
        self.final_fc = nn.Linear(hidden_size, vocab_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self,  metacognitive_vector_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids ,
                       context_attention_mask,
                       student_attention_mask):

        metacognitive_vector_emb = self.metacognitive_emb(metacognitive_vector_ids)

        context_encoding = self.context_encoder(problem_expected_code_ids,
                                                attention_masks=context_attention_mask)
        
        seq_length = problem_student_code_ids.size(1)
        token_embeds = self.token_embedding(problem_student_code_ids)
        position_ids = torch.arange(0, seq_length, device=problem_student_code_ids.device).unsqueeze(0)
        position_embeds = self.position_embedding(position_ids)
        
        # Combine embeddings
        inputs_embeds = token_embeds + position_embeds
        inputs_embeds = self.dropout(inputs_embeds)
        
     
        # outputs = self.gpt2_model(inputs_embeds=inputs_embeds, attention_mask=student_attention_mask, output_hidden_states=True)
        # hR = (outputs.hidden_states)[-1]
        # # print("hR shape",hR.shape)
      
        transformer_input = inputs_embeds
        for transformer_block in self.transformer_blocks:
            transformer_output = transformer_block(transformer_input, metacognitive_vector_emb, context_encoding)           
       
        logits = self.final_fc(transformer_output)    

        return logits

In [ ]:
model = PAAModel()

In [ ]:
model.to(device)

In [50]:
torch.cuda.empty_cache()

In [51]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [52]:
LOSS = torch.nn.CrossEntropyLoss()

In [53]:
num_epochs = 200

In [54]:
df_train = df[0:400]

In [55]:
len(df)

500

In [56]:
from torch.utils.data import DataLoader
train_dataset = CustomDataset(df_train, t5_tokenizer , gpt2_tokenizer)
train_dataloader = DataLoader(train_dataset , batch_size = 1 ,shuffle = True )

In [57]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [58]:
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

In [75]:
for epoch in range(num_epochs):
    print(f"Training started for epoch {epoch + 1}/{num_epochs}")
    model.train()
    total_loss = 0

    for idx, (metacognition_vector_ids,
              problem_student_code_ids,
              problem_expected_code_ids,
              student_code_ids,
              target_ids) in enumerate(train_dataloader):
        
        metacognition_vector_ids = metacognition_vector_ids.to(device)
        problem_student_code_ids = problem_student_code_ids.to(device)
        problem_expected_code_ids = problem_expected_code_ids.to(device)
        student_code_ids = student_code_ids.to(device)
        target_ids = target_ids.to(device)
        
        #attention masking
        student_attention_mask = (problem_student_code_ids != gpt2_pad_token_id).long().to(device)
        context_attention_mask = (problem_expected_code_ids != t5_pad_token_id).long().to(device)
        

        optimizer.zero_grad()
        logits = model(metacognition_vector_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids ,
                       context_attention_mask,
                       student_attention_mask)

     
        logits = logits.view(-1, logits.size(-1))
        target_ids = target_ids.view(-1)

       
        loss = LOSS(logits, target_ids)
        total_loss += loss.item()

      
        loss.backward()       
        for name, param in model.named_parameters():
            if 'context_encoder' in name or 'persona_encoder' in name:
                assert param.grad is None, f"Gradients found in frozen encoder {name}"
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)       
        optimizer.step()    
        
        if idx % 10 == 0:
            print(f"Batch {idx + 1}/{len(train_dataloader)} | Loss: {loss.item():.4f}" , end='\r')




    
                    
    if epoch % 20 ==0 :
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f"Layer: {name} | Grad Norm: {param.grad.norm().item()}")
                elif param.requires_grad:
                    print(f"Layer: {name} | Grad: None")       
    
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch + 1}.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / max(len(train_dataloader), 1),
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

   
    avg_loss = total_loss / max(len(train_dataloader), 1)
    #writer.add_scalar("Loss/train", avg_loss, epoch + 1)
    print(f"Epoch [{epoch + 1}/{num_epochs}] completed | Average Loss: {avg_loss:.4f}")

Training started for epoch 1/200
Layer: token_embedding.weight | Grad Norm: 8.595009426404232e-11
Layer: position_embedding.weight | Grad Norm: 2.1067354458170762e-11
Layer: self_attention.in_proj_weight | Grad: None
Layer: self_attention.in_proj_bias | Grad: None
Layer: self_attention.out_proj.weight | Grad: None
Layer: self_attention.out_proj.bias | Grad: None
Layer: transformer_blocks.0.self_attention.in_proj_weight | Grad Norm: 3.422449434964392e-09
Layer: transformer_blocks.0.self_attention.in_proj_bias | Grad Norm: 6.052214285290347e-10
Layer: transformer_blocks.0.self_attention.out_proj.weight | Grad Norm: 4.0589096528265145e-09
Layer: transformer_blocks.0.self_attention.out_proj.bias | Grad Norm: 1.0387882776186075e-09
Layer: transformer_blocks.0.context_attn.in_proj_weight | Grad Norm: 1.5659276542123735e-08
Layer: transformer_blocks.0.context_attn.in_proj_bias | Grad Norm: 9.279760071478904e-09
Layer: transformer_blocks.0.context_attn.out_proj.weight | Grad Norm: 1.8311679284

KeyboardInterrupt: 

In [60]:
torch.save(model.state_dict(), 'paamodel.pth')

In [61]:
model = PAAModel()

# Load the saved state dict
model.load_state_dict(torch.load('paamodel.pth'))

<ipython-input-61-576f8403c810>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('paamodel.pth'))


<All keys matched successfully>

In [68]:
df_eval = df[149:150].reset_index(drop=True)

In [ ]:
df_eval

In [69]:
eval_dataset = CustomDataset(df_eval, t5_tokenizer,gpt2_tokenizer)

In [65]:


def inference(model,gpt2_tokenizer, t5_tokenizer, eval_dataset, device):
    model.eval()
    model.to(device) 

    metacognitive_vector_ids, problem_student_code_ids, problem_expected_code_ids,student_code_ids, target_ids = eval_dataset[0]

    metacognitive_tensor = metacognitive_vector_ids.unsqueeze(0).to(device)  
    problem_student_code_tensor = problem_student_code_ids.unsqueeze(0).to(device)
    problem_expected_code_tensor = problem_expected_code_ids.unsqueeze(0).to(device)
    target_tensor = target_ids.unsqueeze(0).to(device)

    student_attention_mask = (problem_student_code_tensor != gpt2_tokenizer.pad_token_id).long().to(device)
    context_attention_mask = (problem_expected_code_tensor != t5_tokenizer.pad_token_id).long().to(device)

    
    with torch.no_grad():         
        
        logits = model(
            metacognitive_vector_ids=metacognitive_tensor,
            problem_student_code_ids=problem_student_code_tensor,
            problem_expected_code_ids=problem_expected_code_tensor,
            context_attention_mask=context_attention_mask,
            student_attention_mask=student_attention_mask
        )
        
        predictions = logits.argmax(dim=-1).squeeze().tolist()  
        decoded_text = gpt2_tokenizer.decode(predictions, skip_special_tokens=True)

        
        return predictions, decoded_text



In [70]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset, device)

In [ ]:
print("Predicted Tokens:", predictions)

In [71]:
print("Decoded Text:", decoded_text)

Decoded Text: Your approach to reading the problem entirely before starting is excellent. However, you may benefit from summarizing the problem in your own words and identifying the key points more frequently. This will help you better understand the requirements, especially when dealing with complex inputs like arrays of strings or arrays of arrays. For this problem, breaking down the task into smaller sub-goals would have been beneficial. For instance, first handling single strings, then arrays of two strings with the appropriate prefixes, and finally combining all elements into the final string. This breakdown could help you manage the different types of inputs more systematically. Before diving into coding, try to sketch out the algorithm or plan the solution. This planning step can prevent errors like the one in your code where the variable names are inconsistent (you used `item` instead of `a`). Being more vigilant during the implementation process and verifying intermediate resu

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer=None):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded: Epoch {epoch}, Loss: {loss:.4f}")
    return model, optimizer, epoch, loss

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_89.pth"
model, optimizer, start_epoch, _ = load_checkpoint(checkpoint_path, model, optimizer)

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_10.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Use GPU if available: 'cuda'
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
len(df)

In [62]:
df_eval1 = df[449:450].reset_index(drop=True)

In [63]:
eval_dataset1 = CustomDataset(df_eval1, t5_tokenizer,gpt2_tokenizer)

In [66]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset1, device)

In [67]:
print("Decoded Text:", decoded_text)

Decoded Text:  diving on your problem problem diving yous track's crucial's on steps few few you diving you You and you and crucial implementation you, you you. you which second identify you identify and,,,,, and of you you the and, on, the,,, you you solution attention coding coding plan a in and or,, you a plan in logical algorithm you in a instance and, coding with diving in the.,, each in algorithm plan with a or ensure as, needed needed you. and algorithm with, a. algorithm the and mistakes manually through, plan first correct you coding through and during is a This ( and a sketch the the diving a have plan the you with you list Think you the you a a before the will as you you needed coding are each a the the the the plan the needed the the, and you
 and track.. and are you a implementation each., the the a This systematically
 your. the list sequence ensure and the coding solutionolving meets. your the your the that the the the as and the. to,. the each each the coding the the wi